In [66]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, classification_report,
    mean_squared_error, r2_score, f1_score
)
from sklearn.preprocessing import OneHotEncoder
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.decomposition import TruncatedSVD



In [67]:
df = pd.read_csv('../dados_limpos/dados.csv')


In [68]:

df['DatGeracaoConjuntoDados'] = pd.to_datetime(df['DatGeracaoConjuntoDados'], errors='coerce')
df['DatVencimentoTitulo']     = pd.to_datetime(df['DatVencimentoTitulo'], errors='coerce')
df['DatIncidenciaMultaMora']  = pd.to_datetime(df['DatIncidenciaMultaMora'], errors='coerce')

df['Codcvnarr']      = pd.to_numeric(df['Codcvnarr'], errors='coerce').astype('Int64')

df['NumCPFCNPJ'] = (
    df['NumCPFCNPJ']
    .astype(str)
    .str.replace(r'\D+', '', regex=True)
    .replace({'', 'nan', 'None'}, np.nan)
)

string_cols = [
    'AnmArrecadacao',
    'SigNomAgente',
    'DscSituacaoArrecadacao',
    'DscSituacaoCredito'
]

for col in string_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .replace({'nan': np.nan, 'None': np.nan})
    )

for col in string_cols:
    df[col] = df[col].astype('category')

    
df['QtdDiasEmAtraso'] = (
    pd.to_numeric(df['QtdDiasEmAtraso'], errors='coerce')
      .astype('Int64')
)

def to_float_br(series):
    return (series.astype(str)
                  .str.strip()
                  .str.replace('.', '', regex=False)
                  .str.replace(',', '.', regex=False)
                  .replace(['', 'nan', 'None'], np.nan)
                  .astype(float))

df['VlrPcpPrvArr']      = to_float_br(df['VlrPcpPrvArr'])
df['VlrTotPvrArr']      = to_float_br(df['VlrTotPvrArr'])
df['VlrTotPagArr']      = to_float_br(df['VlrTotPagArr'])
df['VlrTotDifPvrPagArr'] = to_float_br(df['VlrTotDifPvrPagArr'])
df['VlrSelic']          = to_float_br(df['VlrSelic'])

df['VlrPcpPrvArr']       = df['VlrPcpPrvArr'].round(2)
df['VlrTotPvrArr']       = df['VlrTotPvrArr'].round(2)
df['VlrTotPagArr']       = df['VlrTotPagArr'].round(2)
df['VlrTotDifPvrPagArr'] = df['VlrTotDifPvrPagArr'].round(2)
df['VlrSelic']           = df['VlrSelic'].round(2)

In [69]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415065 entries, 0 to 415064
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   DatGeracaoConjuntoDados  415065 non-null  datetime64[ns]
 1   Codcvnarr                415065 non-null  Int64         
 2   AnmArrecadacao           415065 non-null  category      
 3   DatVencimentoTitulo      415065 non-null  datetime64[ns]
 4   DatIncidenciaMultaMora   415065 non-null  datetime64[ns]
 5   QtdDiasEmAtraso          415065 non-null  Int64         
 6   NumCPFCNPJ               415065 non-null  object        
 7   SigNomAgente             415065 non-null  category      
 8   DscSituacaoArrecadacao   415065 non-null  category      
 9   DscSituacaoCredito       415065 non-null  category      
 10  VlrPcpPrvArr             415065 non-null  float64       
 11  VlrTotPvrArr             415065 non-null  float64       
 12  VlrTotPagArr    

In [70]:
df.columns


Index(['DatGeracaoConjuntoDados', 'Codcvnarr', 'AnmArrecadacao',
       'DatVencimentoTitulo', 'DatIncidenciaMultaMora', 'QtdDiasEmAtraso',
       'NumCPFCNPJ', 'SigNomAgente', 'DscSituacaoArrecadacao',
       'DscSituacaoCredito', 'VlrPcpPrvArr', 'VlrTotPvrArr', 'VlrTotPagArr',
       'VlrTotDifPvrPagArr', 'VlrSelic', 'AnoArrec', 'MesArrec', 'fatura_paga',
       'fatura_atrasado', 'fatura_nao_paga', 'TrimestreVencimento'],
      dtype='object')

In [71]:
df['prop_pago'] = df['VlrTotPagArr'] / df['VlrTotPvrArr']
df['prop_pago'] = df['prop_pago'].fillna(0)


### Implementação dos modelos de machine learning

# primeiro

In [72]:
cols_to_drop = [
    'fatura_atrasado',
    'fatura_paga',
    'fatura_nao_paga',
    'QtdDiasEmAtraso',
    'VlrTotPvrArr',
    'VlrTotPagArr',
    'VlrTotDifPvrPagArr',
    'DscSituacaoArrecadacao',
    'DscSituacaoCredito',
    'Codcvnarr',
    'NumCPFCNPJ',
    'DatIncidenciaMultaMora',
    'DatVencimentoTitulo',
    'DatGeracaoConjuntoDados'
]

X = df.drop(columns=cols_to_drop)
y = df['fatura_atrasado']


In [73]:
numerical = X.select_dtypes(include=['float64','int64','Int64']).columns
categorical = X.select_dtypes(include=['category']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical)
    ]
)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

In [75]:
log_reg = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', LogisticRegression(max_iter=100))
])

log_reg.fit(X_train, y_train)
pred = log_reg.predict(X_test)

print("Acurácia baseline:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))
f1 = f1_score(y_test, pred)

Acurácia baseline: 0.9908448074397986
              precision    recall  f1-score   support

           0       0.83      0.52      0.64      1300
           1       0.99      1.00      1.00     81713

    accuracy                           0.99     83013
   macro avg       0.91      0.76      0.82     83013
weighted avg       0.99      0.99      0.99     83013



In [76]:
log_reg = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', LogisticRegression(max_iter=150))
])

log_reg.fit(X_train, y_train)
pred = log_reg.predict(X_test)

print("Acurácia baseline:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))
f1 = f1_score(y_test, pred)

Acurácia baseline: 0.9908448074397986
              precision    recall  f1-score   support

           0       0.83      0.52      0.64      1300
           1       0.99      1.00      1.00     81713

    accuracy                           0.99     83013
   macro avg       0.91      0.76      0.82     83013
weighted avg       0.99      0.99      0.99     83013



In [77]:
log_reg = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', LogisticRegression(max_iter=200))
])

log_reg.fit(X_train, y_train)
pred = log_reg.predict(X_test)

print("Acurácia baseline:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))
f1 = f1_score(y_test, pred)

Acurácia baseline: 0.9908448074397986
              precision    recall  f1-score   support

           0       0.83      0.52      0.64      1300
           1       0.99      1.00      1.00     81713

    accuracy                           0.99     83013
   macro avg       0.91      0.76      0.82     83013
weighted avg       0.99      0.99      0.99     83013



In [78]:
result = permutation_importance(
    log_reg, X_test, y_test, n_repeats=5, random_state=42
)

importances = result.importances_mean
indices = np.argsort(importances)[::-1]

for idx in indices[:20]:
    print(X.columns[idx], importances[idx])

SigNomAgente 0.017551467842386193
VlrSelic 0.001214267644826772
AnoArrec 0.000399937359208824
AnmArrecadacao 0.0003517521352077946
VlrPcpPrvArr 0.00014937419440332888
TrimestreVencimento 3.854817920088127e-05
prop_pago 0.0
MesArrec -6.0231530001253476e-05


In [79]:
rf_cls = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', RandomForestClassifier(n_estimators=100))
])

rf_cls.fit(X_train, y_train)
pred_rf = rf_cls.predict(X_test)

print("Acurácia RF:", accuracy_score(y_test, pred_rf))
print(classification_report(y_test, pred_rf))
print("F1 Score RF:", f1_score(y_test, pred_rf))
f1 = f1_score(y_test, pred_rf)

Acurácia RF: 0.996952284581933
              precision    recall  f1-score   support

           0       0.96      0.84      0.90      1300
           1       1.00      1.00      1.00     81713

    accuracy                           1.00     83013
   macro avg       0.98      0.92      0.95     83013
weighted avg       1.00      1.00      1.00     83013

F1 Score RF: 0.9984534979675418


In [80]:
rf_cls = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', RandomForestClassifier(n_estimators=150))
])

rf_cls.fit(X_train, y_train)
pred_rf = rf_cls.predict(X_test)

print("Acurácia RF:", accuracy_score(y_test, pred_rf))
print(classification_report(y_test, pred_rf))
print("F1 Score RF:", f1_score(y_test, pred_rf))
f1 = f1_score(y_test, pred_rf)

Acurácia RF: 0.9968438678279306
              precision    recall  f1-score   support

           0       0.96      0.83      0.89      1300
           1       1.00      1.00      1.00     81713

    accuracy                           1.00     83013
   macro avg       0.98      0.92      0.95     83013
weighted avg       1.00      1.00      1.00     83013

F1 Score RF: 0.9983984742716726


In [81]:
rf_cls = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', RandomForestClassifier(n_estimators=180))
])

rf_cls.fit(X_train, y_train)
pred_rf = rf_cls.predict(X_test)

print("Acurácia RF:", accuracy_score(y_test, pred_rf))
print(classification_report(y_test, pred_rf))
print("F1 Score RF:", f1_score(y_test, pred_rf))
f1 = f1_score(y_test, pred_rf)

Acurácia RF: 0.9969161456639322
              precision    recall  f1-score   support

           0       0.96      0.84      0.89      1300
           1       1.00      1.00      1.00     81713

    accuracy                           1.00     83013
   macro avg       0.98      0.92      0.95     83013
weighted avg       1.00      1.00      1.00     83013

F1 Score RF: 0.998435169564048


In [82]:
result = permutation_importance(
    rf_cls, X_test, y_test, n_repeats=5, random_state=42
)

importances = result.importances_mean
indices = np.argsort(importances)[::-1]

for idx in indices[:20]:
    print(X.columns[idx], importances[idx])

SigNomAgente 0.026728343753388017
VlrSelic 0.009345524195005606
AnoArrec 0.002681507715658982
VlrPcpPrvArr 0.001746714370038438
AnmArrecadacao 0.0016840735788370641
prop_pago 0.0
MesArrec -7.2277836001610755e-06
TrimestreVencimento -4.095744040091276e-05


-- Conjunto de parametros reduzidos para diminuir o tempo do treinamento

In [ ]:
param_grid_rf = {
    "model__n_estimators": [150, 250],#
    "model__max_depth": [None, 5, 10],
    "model__min_samples_split": [2, 5],
    "model__min_samples_leaf": [1, 2],
    "model__max_features": ["sqrt"]
}


grid_rf = GridSearchCV(
    estimator=Pipeline(steps=[
        ('prep', preprocessor),
        ('model', RandomForestClassifier(random_state=42))
    ]),
    param_grid=param_grid_rf,
    scoring="f1",
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_rf.fit(X_train, y_train)

print("Melhores hiperparâmetros:", grid_rf.best_params_)

Otimização de hiperparâmetros com randomForest foi muito pesada para executar e não foi possível esperar o experimento terminar de executar

In [84]:
xgb_cls = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', XGBClassifier(
        eval_metric='logloss',
        tree_method='hist'
    ))
])

xgb_cls.fit(X_train, y_train)
pred_xgb = xgb_cls.predict(X_test)

print("Acurácia XGBClassifier:", accuracy_score(y_test, pred_xgb))
print(classification_report(y_test, pred_xgb))

Acurácia XGBClassifier: 0.9946634864418826
              precision    recall  f1-score   support

           0       0.96      0.69      0.80      1300
           1       1.00      1.00      1.00     81713

    accuracy                           0.99     83013
   macro avg       0.98      0.85      0.90     83013
weighted avg       0.99      0.99      0.99     83013



In [85]:
from sklearn.model_selection import GridSearchCV

param_grid_xgb = {
    "model__n_estimators": [200],
    "model__max_depth": [4, 8],
    "model__learning_rate": [0.03, 0.1],
    "model__subsample": [0.8, 1.0],
    "model__colsample_bytree": [0.8]
}

grid_xgb = GridSearchCV(
    estimator=Pipeline(steps=[
        ('prep', preprocessor),
        ('model', XGBClassifier(
            eval_metric='logloss',
            tree_method='hist'
        ))
    ]),
    param_grid=param_grid_xgb,
    scoring='f1',
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_xgb.fit(X_train, y_train)

print("Melhores hiperparâmetros XGB:", grid_xgb.best_params_)

best_xgb_reg = grid_xgb.best_estimator_

pred_xgb_tuned = best_xgb_reg.predict(X_test)

print("MSE após tuning:", mean_squared_error(y_test, pred_xgb_tuned))
print("R² após tuning:", r2_score(y_test, pred_xgb_tuned))

Fitting 3 folds for each of 8 candidates, totalling 24 fits


Melhores hiperparâmetros XGB: {'model__colsample_bytree': 0.8, 'model__learning_rate': 0.1, 'model__max_depth': 8, 'model__n_estimators': 200, 'model__subsample': 1.0}
MSE após tuning: 0.004878753930107333
R² após tuning: 0.6835051667703755


In [86]:
cv_scores_xgb = cross_val_score(
    best_xgb_reg,
    X_train,
    y_train,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)


print("MSE médio (CV):", cv_scores_xgb.mean())
print("Desvio padrão:", cv_scores_xgb.std())

MSE médio (CV): -0.005746087962126414
Desvio padrão: 0.00018024228744226807


# segundo

In [87]:
colunas_para_remover = [
    'VlrTotDifPvrPagArr',
    'VlrTotPvrArr',
    'VlrTotPagArr',
    'fatura_paga',
    'fatura_atrasado',
    'fatura_nao_paga',
    'NumCPFCNPJ',
    'DatGeracaoConjuntoDados',
    'DatIncidenciaMultaMora',
    'AnoArrec',
    'MesArrec',
    'TrimestreVencimento',
]
df_sample = df.sample(frac=0.20, random_state=42)

X2 = df_sample.drop(columns=colunas_para_remover)
y2 = df_sample['VlrTotDifPvrPagArr']

In [88]:
numerical = X2.select_dtypes(include=['int64', 'float64']).columns
categorical = X2.select_dtypes(include=['object', 'category']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical),
        ('cat', Pipeline([
            ('onehot', OneHotEncoder(handle_unknown='ignore')),
            ('svd', TruncatedSVD(n_components=100))
        ]), categorical)
    ]
)

In [89]:
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=42
)

In [90]:
lin_reg = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', LinearRegression())
])

lin_reg.fit(X2_train, y2_train)
pred = lin_reg.predict(X2_test)

print("MSE (LinearRegression):", mean_squared_error(y2_test, pred))
print("R² (LinearRegression):", r2_score(y2_test, pred))


MSE (LinearRegression): 2.74667122514781e+30
R² (LinearRegression): 0.00435028856654085


In [91]:
result = permutation_importance(
    lin_reg, X2_test, y2_test, n_repeats=5, random_state=42
)

importances = result.importances_mean
indices = np.argsort(importances)[::-1]

for idx in indices[:20]:
    print(X2.columns[idx], importances[idx])

AnmArrecadacao 0.006135601598733298
DscSituacaoArrecadacao 0.00415043981819776
prop_pago 0.0018516406113094019
Codcvnarr 0.0015971956412008214
DscSituacaoCredito 0.0012370301834981668
QtdDiasEmAtraso 0.0008574186092599634
SigNomAgente 0.00039449018970989514
VlrSelic 1.6161048811635937e-06
DatVencimentoTitulo 0.0
VlrPcpPrvArr -1.2026813354704124e-05


In [92]:
xgb_reg = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', XGBRegressor(tree_method='hist', max_depth=5, learning_rate=0.1))
])

xgb_reg.fit(X2_train, y2_train)
pred_xgb = xgb_reg.predict(X2_test)

print("MSE (XGBRegressor):", mean_squared_error(y2_test, pred_xgb))
print("R² (XGBRegressor):", r2_score(y2_test, pred_xgb))

MSE (XGBRegressor): 2.576641484415938e+30
R² (XGBRegressor): 0.06598491769317916


In [93]:
result = permutation_importance(
    xgb_reg, X2_test, y2_test, n_repeats=5, random_state=42
)

importances = result.importances_mean
indices = np.argsort(importances)[::-1]

for idx in indices[:20]:
    print(X2.columns[idx], importances[idx])

AnmArrecadacao 0.15984088250823475
QtdDiasEmAtraso 0.11139402785265366
prop_pago 0.08355404599794751
VlrSelic 0.03701382984782779
SigNomAgente 0.016789740025301248
DscSituacaoArrecadacao 0.014764477805668298
DscSituacaoCredito 0.009161614844707766
VlrPcpPrvArr 0.005295209403094114
Codcvnarr 0.0038077656701439146
DatVencimentoTitulo 0.0


In [94]:

param_grid_xgb_reg = {
    "model__n_estimators": [200, 400],
    "model__max_depth": [4, 6],
    "model__learning_rate": [0.05, 0.1],
    "model__subsample": [0.8],
    "model__colsample_bytree": [0.8]
}

grid_xgb_reg = GridSearchCV(
    estimator=Pipeline(steps=[
        ('prep', preprocessor),
        ('model', XGBRegressor(
            tree_method='hist',
            eval_metric='rmse'
        ))
    ]),
    param_grid=param_grid_xgb_reg,
    scoring='neg_mean_squared_error',
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_xgb_reg.fit(X2_train, y2_train)

print("Melhores hiperparâmetros XGBRegressor:", grid_xgb_reg.best_params_)
best_xgb_reg = grid_xgb_reg.best_estimator_

pred_xgb_tuned = best_xgb_reg.predict(X2_test)

print("MSE após tuning:", mean_squared_error(y2_test, pred_xgb_tuned))
print("R² após tuning:", r2_score(y2_test, pred_xgb_tuned))


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Melhores hiperparâmetros XGBRegressor: {'model__colsample_bytree': 0.8, 'model__learning_rate': 0.05, 'model__max_depth': 4, 'model__n_estimators': 200, 'model__subsample': 0.8}
MSE após tuning: 2.4861021377495817e+30
R² após tuning: 0.09880481748911507


In [95]:
cv_scores_xgb = cross_val_score(
    best_xgb_reg,
    X2_train,
    y2_train,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

print("MSE médio (CV):", -cv_scores_xgb.mean())
print("Desvio padrão:", cv_scores_xgb.std())


MSE médio (CV): 2.4600088284101527e+30
Desvio padrão: 2.7182482248800268e+29


# Terceiro

In [96]:
colunas = [
    'VlrTotPagArr',
    'VlrTotDifPvrPagArr',
    'prop_pago',
    'fatura_paga',
    'fatura_atrasado',
    'fatura_nao_paga',
    'DscSituacaoCredito',
    'DscSituacaoArrecadacao',
    'DatIncidenciaMultaMora',
    'DatGeracaoConjuntoDados',
    'AnoArrec',
    'MesArrec',
    'NumCPFCNPJ',
    'QtdDiasEmAtraso'
]
y3 = df['QtdDiasEmAtraso']
X3 = df.drop(columns=colunas)

X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, y3, test_size=0.2, random_state=42
)

In [97]:
numerical = X3.select_dtypes(include=['float64','int64','Int64']).columns
categorical = X3.select_dtypes(include=['category']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical)
    ]
)

In [98]:
df.columns

Index(['DatGeracaoConjuntoDados', 'Codcvnarr', 'AnmArrecadacao',
       'DatVencimentoTitulo', 'DatIncidenciaMultaMora', 'QtdDiasEmAtraso',
       'NumCPFCNPJ', 'SigNomAgente', 'DscSituacaoArrecadacao',
       'DscSituacaoCredito', 'VlrPcpPrvArr', 'VlrTotPvrArr', 'VlrTotPagArr',
       'VlrTotDifPvrPagArr', 'VlrSelic', 'AnoArrec', 'MesArrec', 'fatura_paga',
       'fatura_atrasado', 'fatura_nao_paga', 'TrimestreVencimento',
       'prop_pago'],
      dtype='object')

In [99]:
lin_reg2 = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', LinearRegression())
])

lin_reg2.fit(X3_train, y3_train)
pred = lin_reg2.predict(X3_test)

print("MSE (LinearRegression - dias atraso):", mean_squared_error(y3_test, pred))
print("R² (LinearRegression - dias atraso):", r2_score(y3_test, pred))

MSE (LinearRegression - dias atraso): 7666.393753342452
R² (LinearRegression - dias atraso): 0.562489233319342


In [100]:
result = permutation_importance(
    lin_reg2, X3_test, y3_test, n_repeats=5, random_state=42
)

importances = result.importances_mean
indices = np.argsort(importances)[::-1]

for idx in indices[:20]:
    print(X3.columns[idx], importances[idx])

AnmArrecadacao 4.026191080136604
Codcvnarr 2.1946993303403124
SigNomAgente 0.9819693884787712
VlrSelic 0.49596723423310884
VlrPcpPrvArr 0.002290048882192175
VlrTotPvrArr 0.0013191213148431747
TrimestreVencimento 0.0007060267707272195
DatVencimentoTitulo 0.0


In [101]:
xgb_reg2 = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', XGBRegressor(tree_method='hist', max_depth=5, learning_rate=0.1))
])

xgb_reg2.fit(X3_train, y3_train)
pred = xgb_reg2.predict(X3_test)

print("MSE (XGBRegressor - dias atraso):", mean_squared_error(y3_test, pred))
print("R² (XGBRegressor - dias atraso):", r2_score(y3_test, pred))

MSE (XGBRegressor - dias atraso): 7038.85302734375
R² (XGBRegressor - dias atraso): 0.5983021855354309


In [102]:
result = permutation_importance(
    xgb_reg2, X3_test, y3_test, n_repeats=5, random_state=42
)

importances = result.importances_mean
indices = np.argsort(importances)[::-1]

for idx in indices[:20]:
    print(X3.columns[idx], importances[idx])

SigNomAgente 0.9219543099403381
Codcvnarr 0.12213364839553834
VlrSelic 0.05116314888000488
VlrPcpPrvArr 0.04076278209686279
VlrTotPvrArr 0.011851012706756592
AnmArrecadacao 0.00501859188079834
TrimestreVencimento 0.0018513917922973633
DatVencimentoTitulo 0.0


In [103]:

param_grid_xgb_reg = {
    "model__n_estimators": [200, 400],
    "model__max_depth": [4, 6],
    "model__learning_rate": [0.05, 0.1],
    "model__subsample": [0.8],
    "model__colsample_bytree": [0.8]
}

grid_xgb_reg = GridSearchCV(
    estimator=Pipeline(steps=[
        ('prep', preprocessor),
        ('model', XGBRegressor(
            tree_method='hist',
            eval_metric='rmse'
        ))
    ]),
    param_grid=param_grid_xgb_reg,
    scoring='neg_mean_squared_error',
    cv=3,
    n_jobs=-1,
    verbose=1
)

In [104]:
grid_xgb_reg.fit(X3_train, y3_train)

print("Melhores hiperparâmetros XGBRegressor:", grid_xgb_reg.best_params_)
best_xgb_reg = grid_xgb_reg.best_estimator_

pred_xgb_tuned = best_xgb_reg.predict(X3_test)

print("MSE após tuning:", mean_squared_error(y3_test, pred_xgb_tuned))
print("R² após tuning:", r2_score(y3_test, pred_xgb_tuned))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Melhores hiperparâmetros XGBRegressor: {'model__colsample_bytree': 0.8, 'model__learning_rate': 0.1, 'model__max_depth': 6, 'model__n_estimators': 400, 'model__subsample': 0.8}
MSE após tuning: 3353.349365234375
R² após tuning: 0.8086289167404175


In [105]:
cv_scores_xgb = cross_val_score(
    best_xgb_reg,
    X3_train,
    y3_train,
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

print("MSE médio (CV):", -cv_scores_xgb.mean())
print("Desvio padrão:", cv_scores_xgb.std())

MSE médio (CV): 3752.4252115885415
Desvio padrão: 31.582435437249558
